# Weather forecasting with Machine Learning

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Load the dataset
weather_df = pd.read_csv('weatherHistory.csv')

In [ ]:
weather_df.head()

,Temperature (C),Humidity,Summary
0,9.472222,0.89,Partly Cloudy
1,9.355556,0.86,Partly Cloudy
2,9.377778,0.89,Mostly Cloudy
3,8.288889,0.83,Partly Cloudy
4,8.755556,0.83,Mostly Cloudy


In [ ]:
weather_df.info()

In [ ]:
weather_df.keys()

Index(['Temperature (C)', 'Humidity', 'Summary'], dtype='object')

In [ ]:
weather_df['Summary'].unique()

array(['Partly Cloudy', 'Mostly Cloudy', 'Overcast', 'Foggy',
       'Breezy and Mostly Cloudy', 'Clear', 'Breezy and Partly Cloudy',
       'Breezy and Overcast', 'Humid and Mostly Cloudy',
       'Humid and Partly Cloudy', 'Windy and Foggy', 'Windy and Overcast',
       'Breezy and Foggy', 'Windy and Partly Cloudy', 'Breezy',
       'Dry and Partly Cloudy', 'Windy and Mostly Cloudy',
       'Dangerously Windy and Partly Cloudy', 'Dry', 'Windy',
       'Humid and Overcast', 'Light Rain', 'Drizzle', 'Windy and Dry',
       'Dry and Mostly Cloudy', 'Breezy and Dry', 'Rain'], dtype=object)

In [ ]:
#weather_df['Summary'].unique()
factorized_mapping = dict(enumerate(weather_df['Summary'].unique()))
weather_df['Summary'] = pd.factorize(weather_df['Summary'])[0]
print(factorized_mapping)

{0: 'Partly Cloudy', 1: 'Mostly Cloudy', 2: 'Overcast', 3: 'Foggy', 4: 'Breezy and Mostly Cloudy', 5: 'Clear', 6: 'Breezy and Partly Cloudy', 7: 'Breezy and Overcast', 8: 'Humid and Mostly Cloudy', 9: 'Humid and Partly Cloudy', 10: 'Windy and Foggy', 11: 'Windy and Overcast', 12: 'Breezy and Foggy', 13: 'Windy and Partly Cloudy', 14: 'Breezy', 15: 'Dry and Partly Cloudy', 16: 'Windy and Mostly Cloudy', 17: 'Dangerously Windy and Partly Cloudy', 18: 'Dry', 19: 'Windy', 20: 'Humid and Overcast', 21: 'Light Rain', 22: 'Drizzle', 23: 'Windy and Dry', 24: 'Dry and Mostly Cloudy', 25: 'Breezy and Dry', 26: 'Rain'}


In [ ]:
# Define features and labels
features = weather_df[['Temperature (C)','Humidity']]
labels = weather_df[['Summary']]

# Train Test Split
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.3, random_state=0)

In [ ]:
# Build a linear regression model using Keras
model = keras.Sequential([
    keras.layers.Dense(units=1, input_shape=(2,))  # Two input features, single output unit
])

# Compile the model
model.compile(optimizer='Adam', loss='mean_squared_error')  # Using Stochastic Gradient Descent (SGD) optimizer

# Train the model
model.fit(
    features_train,labels_train,
    epochs=6,
    validation_data=(features_test, labels_test)
    )




Epoch 1/6
2110/2110 [==============================] - 3s 1ms/step - loss: 4.0173 - val_loss: 3.7153
Epoch 2/6
2110/2110 [==============================] - 3s 1ms/step - loss: 3.8235 - val_loss: 3.7084
Epoch 3/6
2110/2110 [==============================] - 3s 1ms/step - loss: 3.8204 - val_loss: 3.7079
Epoch 4/6
2110/2110 [==============================] - 3s 1ms/step - loss: 3.8184 - val_loss: 3.7039
Epoch 5/6
2110/2110 [==============================] - 3s 2ms/step - loss: 3.8160 - val_loss: 3.7033
Epoch 6/6
2110/2110 [==============================] - 3s 1ms/step - loss: 3.8152 - val_loss: 3.7041


In [ ]:
predictions = model.predict(features_test)
print(predictions)


905/905 [==============================] - 1s 755us/step
[[1.8265911]
 [1.6026549]
 [1.2416341]
 ...
 [2.0215707]
 [2.026103 ]
 [1.6595079]]


In [ ]:
rounded_predictions = [round(prediction[0]) for prediction in predictions]
original_predictions = [factorized_mapping[prediction] for prediction in rounded_predictions]
original_predictions

In [ ]:
rmse = np.sqrt(((labels_test - predictions) ** 2).mean())
print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): Summary    1.924595
dtype: float64


# Convert the Trained Model to Tensor Flow Lite

In [ ]:
import tensorflow as tf

# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("weather_model.tflite", "wb").write(tflite_model)

import os
basic_model_size = os.path.getsize("weather_model.tflite")
print("Model is %d bytes" % basic_model_size)

Model is 1112 bytes


# Encode the Model in an Arduino Header File

In [ ]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat weather_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

Header file, model.h, is 6,892 bytes.

Open the side panel (refresh if needed). Double click model.h to download the file.
